# Exercices Python

## 1. Expressions régulières

Les **expressions régulières** sont des outils particulièrement puissants. Si leur syntaxe peut paraître complexe, le pouvoir d'extraction que leur maîtrise vous conférera vaut largement l'effort. Les expressions régulières sont particulièrement utiles lorsqu'il s'agit de chercher une expression structurée précise dans un texte. Ce texte peut être un autre code (e.g. une page web Wikipedia), ou une source historique, comme un article de journal, ou un dictionnaire historique.

Exemple :

In [ ]:
from re import *
findall('[0-9]+', 'Avenue d’Ouchy 15, 1006 Lausanne')

Dans cet exemple, l'expression régulière ```[0-9]+``` permet d'extraire les nombres d'une
chaîne de caractères (pour ensuite les transformer en ```int``` par exemple). En réalité, cette expression signifie : *Sélectionner un ou plusieurs (+) caractères se suivant et faisant partie de l'ensemble ```[...]``` des caractères compris entre 0 et 9 (```0-9```).*

* Les crochets ```[...]``` permettent de définir un ensemble non-ordonné de caractères, au choix.
* Le tiret ```-``` permet de signifier que l'on cherche un caractère dont le numéro Unicode est compris entre celui du caractère situé juste avant le tiret (e.g. ```0```) et celui du caractère situé juste après le tiret (e.g. ```9```). Il est possible d'obtenir le numéro Unicode d'un caractère à l'aide de la fonction intégrée ```ord()```. Ainsi, la commande ```ord('0')```, ```ord('9')``` nous indique que le caractère ```0``` porte le numéro ```48```, tandis que 9 porte le numéro 57. À titre d'exemple, ```a``` porte le numéro 97, ```à``` le numéro 224 et ```!``` le numéro 33. Veillez donc à toujours vérifier quels caractères se trouvent dans votre intervalle.
* Le signe ```+``` permet d'indiquer que l'on cherche une ou plusieurs occurrences (par exemple un ou plusieurs caractères faisant partie d'un ensemble donné).

Vous pouvez aussi rechercher une séquence précise. Par exemple, vous pourriez vouloir
étudier la présence des femmes nommées dans les journaux lausannois du XIXème siècle.
Pour ce faire, vous pourriez commencer par chercher le mot "Madame" (avec ou sans
majuscule) suivi d'un espace, puis d'un nom commençant par une lettre majuscule. La
syntaxe de votre requête ressemblera alors à ceci :

In [ ]:
text = "Ce matin, Monsieur Larpin fera miser la maison qu'il a acquise de Madame Dupin \
née Mercier."
findall('([Mm]adame )([A-Z][a-zà-ÿ\-]+)', text)

Les parenthèses servent à créer un groupe de caractères. Ici le caractère ```M``` / ```m```, puis la suite de caractères ```"adame "```, dans cet ordre précis. À l'intérieur d'une expression régulière, les caractères spéciaux doivent être échappés. Ici, par exemple, le tiret est échappé (```\-```), pour ne pas être confondu avec son autre fonction, que nous avons découvert dans l'exemple précédent. Notez que les caractères accentués sont distincts des caractères non-accentués et qu'ils doivent donc être inclus expressément (```à–ÿ```).

Mettons maintenant que vous travailliez à une échelle très différente, par exemple sur un
corpus réunissant tous les journaux historiques francophones. Vous pourriez vouloir
rendre cette instruction plus efficace. En réalité, le système des expressions régulières permet de simplement localiser le segment qui vous intéresse, tout en exploitant son contexte à l'aide des "coups d'oeil" : *lookbehind* (```?<=```) et *lookahead* (```?=```). Exemple :

In [ ]:
findall('(?<=[Mm]adame )([A-Z][\-\w]+)', text)

Dans cet exemple, l'expression se trouvant dans la première parenthèse n'est que
contextuelle. Dans la deuxième parenthèse, nous avons utilisé un mécanisme pratique,
avec le raccourci ```\w+``` qui permet de capturer n'importe quel caractère alphabétique.
Prenez le temps d'étudier cette expression régulière pour bien comprendre son
fonctionnement.

Il est aussi possible d'utiliser les expressions régulières pour nettoyer certaines données
ou les remplacer par d'autres. La fonction ```re.sub``` permet de remplacer une expression par une nouvelle chaîne de caractères, un peu à l'image de la fonction *replace*, que vous
connaissez déjà, mais avec la puissance des expressions régulières.

In [ ]:
sub('([^\-\w ]+)', '@', text)

Dans cet exemple, tous les caractères *non-alphabétiques* du texte sont remplacés par un
arobase ```@```. Le signe circonflexe ```^``` permet la négation de l'expression qui suit.
Littéralement, l'expression remplace donc tous les caractères non-alphabétiques, qui ne
sont ni des tirets, ni des espaces. Pour les supprimer, on pourrait remplacer ```@``` par une chaîne de caractères vide.

### Exercice 1

Chargez les données extraites d'Impresso à l'aide du script ci-dessous. Utilisez une expression régulière pour repérer, dans la liste ```articles```, les mentions de personnes au format ```Prénom Nom```.

In [ ]:
import pandas as pd

df = pd.read_csv('impresso_export_jeanne_hersch.csv', sep=';')
articles = df['content'].dropna().values.tolist()[:500]

## 2. Entités nommées

Les **entités nommées** sont des expressions textuelles caractéristiques et reconnaissables : par exemple des noms propres, des noms de lieux, des dates, des noms d'organisations,
etc. La reconnaissance et l'extraction des entités nommées constitue un enjeu majeur du
*natural language processing*. En humanités digitales, ces technologies peuvent être très utiles pour créer des liens entre des données textuelles, des lieux et des personnes.

```
conda install spacy && conda install spacy-transformers && python -m spacy download fr_core_news_lg
```

In [4]:
import spacy
nlp_fr = spacy.load('fr_core_news_lg')
doc = nlp_fr('Grand concert vocal et instrumental dans la salle du Casino, \
le mardi 5, par Mme Pollet, harpiste de Paris, et MM. Girard et Lagoanère;')
[(ent.label_, ent.text) for ent in doc.ents]

[('LOC', 'Casino'),
 ('PER', 'Mme Pollet'),
 ('LOC', 'Paris'),
 ('PER', 'MM. Girard'),
 ('PER', 'Lagoanère')]

Les modèles NLP comme celui de spacy sont en général pré-entraînés sur des corpus très
grands, comme par exemple Wikipedia, ou des corpus de journaux. La reconnaissance des entités nommées est évidemment spécifique à la langue. En conséquence, les modèles pour certaines langues, comme l'anglais ou le chinois, sont très performants, tandis que d'autres le sont nettement moins. Le français se trouve en général dans une tranche intermédiaire.

### Exercice 2

Reprenez les données de l'exercice précédent. Utilisez maintenant spacy pour repérer les entités nommées désignant des personnes dans les articles. Que constatez-vous?

## 3. Latent Dirichlet Allocation

La Latent Dirichlet Allocation (LDA, à ne pas confondre avec la Linear Discriminant Analysis) est une méthode de **topic modelling**, qui permet de distinguer
les différents thèmes d'un texte ou d'une série de textes en s'appuyant sur la distribution
statistique des **tokens** et en particulier sur la co-occurence de certains groupes de tokens.

Pour utiliser LDA sur des données textuelles, il est nécessaire de rendre ces dernières
numériquement intelligibles. La manière la plus simple de faire ceci est de subdiviser le
texte en segments plus petits, que l'on appellera documents. 

Un document peut par
exemple correspondre à une phrase, une réplique, ou à une entrée dans votre base de
données. Chaque document contient un ou plusieurs tokens. L'ensemble des types (ici
l'ensemble des tokens uniques existants) constitue le vocabulaire. Un numéro est attribué
à chaque type du vocabulaire. 

Par exemple, *académie* correspondra au numéro 0, *Acadie*
au numéro 1, *acétone* au numéro 2, et ainsi de suite. Grâce à ce système, chaque document
peut être représenté comme une liste de 0 et de 1. La longueur du vecteur correspondra au nombre de mots compris dans le vocabulaire. Si le document contient un token correspondant au nième type du vocabulaire, la nième entrée de la liste prendra la valeur 1. Tous les autres types, absents du document en question, prendront la valeur 0 (*one-hot encoding*).

Pour commencer, installez la librairie suivante

```
conda install scikit-learn
```

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

documents, tokens = [], []
for article in articles:
    words = findall('([a-zà-ÿ0-9]+)', article.lower())
    tokens += words
    documents.append(words)

Dans cet exemple, nous créons deux listes : **documents** et **tokens**, en itérant très simplement sur chaque article. Nous ne considérons pas la capitalisation des caractères et nous stockons chaque mot dans la liste continue tokens et dans la liste des documents, qui stocke séparément les mots de chaque article dans une sous-liste. Notez qu'il existe des manières plus efficientes de créer ces deux listes avec Python, ici nous avons privilégié la notation la plus lisible.

In [ ]:
vocabulaire = pd.Series(tokens).value_counts()[20:]
vocabulaire = vocabulaire [vocabulaire >= 3].sort_index().keys()

Dans un deuxième temps, nous comptons toutes les occurrences uniques des tokens et ne conservons que ceux qui apparaissent au moins 3 fois. En dessous de ce seuil, l'utilisation d'un modèle statistique est peu pertinente. Nous enlevons également les 20 tokens les plus fréquents, qui correspondent aux mots tels que "de", "la", "et", "à", "les", "des", etc. Cette liste des mots-clés uniques apparaissant au moins 3 fois constitue le **vocabulaire**.

In [ ]:
# Temps de calcul indicatif pour cette cellule : 3 minutes
one_hot = np.zeros((len(documents), len(vocabulaire)))
for index, document in enumerate(documents):
    for token in document:
        one_hot[index][int(np.argmax(vocabulaire == token))] = 1

Pour *chaque* document, une liste de zéros de longueur ```len(vocabulaire)``` est initialisée. Une boucle itère ensuite sur chaque **token *n*** du document et remplace la nième valeur de la liste par 1. n correspond au numéro attribué au token dans le vocabulaire.

À présent, puisque nos données ont été traduites en langage numérique, il est possible
d'appliquer la LDA. La LDA est un algorithme de clustering paramétrique. Il est en effet
nécessaire de lui indiquer le nombre de clusters attendu (ici, nous utiliserons la valeur ```7```). Dans le cas du topic modelling,
chaque cluster correspond intuitivement à un thème.

In [ ]:
lda = LatentDirichletAllocation(n_components=7, random_state=0)
res = lda.fit_transform(one_hot)

Les clusters sont simplement représentés par des numéros. Le travail d'interprétation du *topic* de chaque cluster nous revient. Pour faciliter cette interprétation, nous pouvons par exemple
visualiser les mots les plus fréquents ou les mots les plus caractéristiques de chaque cluster.

In [ ]:
# Mots les plus fréquents du cluster 5
for i in np.flip(np.argsort(lda.components_[5]))[:20]:
    print(vocabulaire[i], end=', ')

In [ ]:
# Mots les plus caractéristiques du cluster 5
type_attribution = lda.transform(np.identity(len(vocabulaire)))
for t in vocabulaire[np.flip(np.argsort(type_attribution[:,5]))[:20]]:
    print(t, end=', ')

Dans ce cas, on remarque un thème centré sur les conférences, et les réceptions, les évènements publics.

On peut essayer d'encore mieux comprendre ce thème en affichant les 3 articles les plus caractéristiques de ce dernier.

In [ ]:
for art_id in np.flip(np.argsort(res[:, 5]))[:3]:
    print(articles[art_id], end='\n\n')

### Exercice 3
Reproduisez l'exercice sur la *Latent Dirichlet Allocation* sur un corpus *Impresso* correspondant à l'un de vos personnages. Arrivez-vous à interpréter les thèmes obtenus ? Qu'est-ce que ces thèmes vous permettent de dire par rapport à la représentation médiatique de ce dernier ?

### Exercice 4
Reproduisez l'exemple sur la reconnaissance des entités nommées sur votre propre corpus. Calculez et affichez les lieux les plus cités.